# 归约运算

归约（Reduction）是将输入张量的某个维度归约为标量的运算，例如 NumPy 的 `np.sum`。使用 for 循环通常可以直接实现归约。但是在 TVM 中有点复杂，因为不能直接使用 Python for 循环。在本节中，将描述如何在 TVM 中实现归约。

In [1]:
# from tvm_book.contrib import d2ltvm
import numpy as np
import tvm
from tvm import te

## 求和

从对二维矩阵的行求和开始把它变成一维向量。在 NumPy 中，可以用 `sum` 方法来完成。

In [2]:
a = np.random.normal(size=(3,4)).astype('float32')
a.sum(axis=1)

array([1.0295496, 3.784973 , 2.0665042], dtype=float32)

和前面一样，从头实现这个运算，以帮助理解 TVM 表达式。

In [3]:
def sum_rows(a, b):
    """a is an n-by-m 2-D matrix, b is an n-length 1-D vector 
    """
    n = len(b)
    for i in range(n):
        b[i] = np.sum(a[i,:])

b = np.empty((3,), dtype='float32')
sum_rows(a, b)
b

array([1.0295496, 3.784973 , 2.0665042], dtype=float32)

这非常简单，首先对第一个维度进行迭代，`axis=0`，然后对第二个维度上的所有元素求和，以写入结果。在 NumPy 中，可以使用 `:` 对该维度上的所有元素进行切片。

可以在 TVM 中实现同样的事情。与 [](ch_vector_add_te) 中的加法运算相比，这里使用了两个新的算子。一个是 `tvm.reduce_axis`，创建范围为 0 到 `m` 的归约轴。它的功能类似于 `sum_rows` 中使用的 `:`，但需要在 TVM 中显式指定范围。另一个是 `tvm.sum`，它对沿归约轴 `k` 的所有元素求和，并返回标量。

In [4]:
n, m = te.var('n'), te.var('m')
A = te.placeholder((n, m), name='a')
j = te.reduce_axis((0, m), name='j')
B = te.compute((n,), lambda i: te.sum(A[i, j], axis=j), name='b')
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
ir_mod["main"]

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, n) {
    b[(i*stride)] = 0f
    for (j, 0, m) {
      b[(i*stride)] = (b[(i*stride)] + a[((i*stride) + (j*stride))])
    }
  }
}

可以看到，生成的伪代码将 `tvm.sum` 扩展成另一个沿着 `k` 轴的 for 循环。正如前面提到的，伪代码是类似于 C 语言的，因此将 `a[i,j]` 的索引展开为 `(i*m)+j`，将 `a` 处理为一维数组。还要注意，`b` 在求和之前被初始化为全零。

现在测试结果与预期一致。

In [5]:
mod = tvm.build(ir_mod)
c = tvm.nd.array(np.empty((3,), dtype='float32'))
mod(tvm.nd.array(a), c)
np.testing.assert_equal(b, c.asnumpy())

`a.sum()` 将对 `a` 中的所有元素求和，并返回标量。TVM 中实现它，需要沿着第一个维度的另一个轴归约，它的大小是 `n`。结果是标量，即 0 阶张量，可以用空元组 `()` 创建。

In [6]:
i = te.reduce_axis((0, n), name='i')
B = te.compute((), lambda: te.sum(A[i, j], axis=(i, j)), name='b')
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
ir_mod["main"]

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  b[0] = 0f
  for (i, 0, n) {
    for (j, 0, m) {
      b[0] = (b[0] + a[((i*stride) + (j*stride))])
    }
  }
}

验证结果：

In [7]:
mod = tvm.build(ir_mod)
c = tvm.nd.array(np.empty((), dtype='float32'))
mod(tvm.nd.array(a), c)
np.testing.assert_allclose(a.sum(), c.asnumpy(), atol=1e-5)

在本例中，使用 `np.testing.assert_allclose` 而不是 `np.testing.assert_equal` 用于验证结果，因为 `float32` 数字的计算可能由于数值误差而不同。

除 `tvm.sum` 之外，在 TVM 中还有其他的归约算子，如 `tvm.min`、`tvm.max`。也可以用它们来实现相应的归约运算。

## 可交换的归约

在数学中，如果 $a\circ b = b\circ a$，则算子 $\circ$ 是可交换的（commutative）。TVM 允许通过 `tvm.comm_reducer` 定义定制的可交换归约算子。它接受两个函数参数，一个定义如何计算，另一个指定初始值。

让我们以行为单位使用乘法，例如 `a.prod(axis=1)`。同样，首先展示如何从头开始实现它。

In [8]:
def prod_rows(a, b):
    """a is an n-by-m 2-D matrix, b is an n-length 1-D vector 
    """
    n, m = a.shape
    for i in range(n):
        b[i] = 1
        for j in range(m):
            b[i] = b[i] * a[i, j]

可以看出，首先需要将返回值初始化为 1，然后使用标量积 `*` 计算归约。现在让我们在 TVM 中定义这两个函数，作为 `te.comm_reducer` 的参数。如前所述，第一个函数定义两个标量输入的 $a\circ b$。第二个参数接受数据类型参数，以返回元素的初始值。然后可以创建归约算子。

In [9]:
comp = lambda a, b: a * b
init = lambda dtype: tvm.tir.const(1, dtype=dtype)
product = te.comm_reducer(comp, init)

`product` 的用法类似于 `te.sum`。实际上，`te.sum` 是预定义的归约算子，使用相同的方法。

In [10]:
n = te.var('n')
m = te.var('m')
A = te.placeholder((n, m), name='a')
k = te.reduce_axis((0, m), name='k')
B = te.compute((n,), lambda i: product(A[i, k], axis=k), name='b')
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
ir_mod["main"]

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, n) {
    b[(i*stride)] = 1f
    for (k, 0, m) {
      b[(i*stride)] = (b[(i*stride)]*a[((i*stride) + (k*stride))])
    }
  }
}

生成的伪代码与按行求和的伪代码类似，只是初始值和归约算法不同。

再来验证一下结果。

In [11]:
mod = tvm.build(ir_mod)
b = tvm.nd.array(np.empty((3,), dtype='float32'))
mod(tvm.nd.array(a), b)
np.testing.assert_allclose(a.prod(axis=1), b.asnumpy(), atol=1e-5)

## 小结

- 可以应用归约算子，例如 `te.sum` 对归约轴 `te.reduce_axis` 求和。
- 可以通过 `te.comm_reducer` 实现定制的可交换归约算子。